In [20]:
# Liturgický kalendár: 365 dní ako "one page" zobrazenie.
# poznámka: po vygenerovaní html súboru je potrebné ešte manuálne vložiť javascript a štýlovanie.

In [21]:
import os
import pandas as pd
from bs4 import BeautifulSoup, Comment

In [22]:
# kde sú html súbory:
data_dir = './data_html/'

In [23]:
# List all html files from data directory 
# with file name length 15 (meaning particular date like: '2023-01-01.html')

# poznámka: 
# keďže cirkevný rok začína adventom, sú v zip archíve aj dni z predošlého roka (2022)
# preto som pre názornosť dal podmienku, aby bral len aktuálny rok 2023 (file.startswith('2023'))
html_files = [ file for file in os.listdir(data_dir) 
                    if file.startswith('2023') and file.endswith(".html") and len(file)==15 ]

len(html_files) 

365

In [24]:
mustra="""
<!DOCTYPE html>
<html lang="sk">
<head>
    <meta charset="UTF-8">
    <title>Liturgický kalendár 2023, skúšobná verzia</title>
    <script></script>
    <style></style>
</head>
<body id="lc">
    <script>init();</script>
</body>
<script>
    naDnes();
</script>
</html>
"""

In [25]:
soup1 = BeautifulSoup(mustra,'lxml')

my_table = soup1.new_tag('table') 
soup1.body.append(my_table)

In [26]:
for fname in html_files:

    with open(data_dir+fname, encoding='utf-8') as f_in:
        s = f_in.read()            

    date_string = fname.removesuffix('.html')

    # týždne sú oddelené hrubšími čiarami, jednotlivé týždne sú zabalené v <tbody> elementoch
    current_date = pd.to_datetime(date_string)
    if (date_string == '2023-01-01') or current_date.day_name('sk_SK')=='Pondelok':
        tbody_here = soup1.new_tag('tbody') # create new tbody here
        soup1.table.append(tbody_here)

    new_table_row = soup1.new_tag('tr')
    tbody_here.append(new_table_row)

    ### soup2
    ### 
    soup2 = BeautifulSoup(s,'lxml') #    
    main = soup2.body.div # div class hvr-scl-chldrn-a

    elems = main.find_all('div', class_='lcHEADinfo') # mažeme súradnice
    for elem in elems: #  v príslušný deň ich môže byť viac
        elem.extract()        
    
    elems = main.find_all('div', class_='lcDENalt') # tu riešime štýlovanie a vigíliu.
    for elem in elems:
        if 'psv' in elem['class']: # rušíme štýlovanie 
            elem['class'].remove('psv') # psv ... prikázaný sviatok, rušíme štýlovanie (červený okraj)

        h3 = elem.find('h3')                       
        if h3:
            if h3.get_text().startswith('Vigília'): # ak je to vigília, mažeme.
                elem.extract()  

    elems = main.find_all('div', class_='lcBODY') # čítania mažeme
    for elem in elems: #  v príslušný deň ich môže byť viac
        elem.extract()        

    # main.find('div', class_='lcNAVIG').extract()    
    main.section.div.extract() # <div class="lcNAVIG"> # navigačné tlačidlá mažeme    

    # h1
    # mažeme link, aj nadbytočný <span> :
    main.h1.a.unwrap() #  lcMY
    main.h1.span.find('span', class_='lcMD').unwrap() # lcMD
    span_lcDMY = main.h1.span.find('span', class_='lcDMY') # lcDMY
    span_lcDMY.smooth()
    span_lcDMY.string = span_lcDMY.string[:-5] # mažeme rok 2023 z textu

    main.h1.span.find('span', class_='lcWD').append(soup2.new_tag('br')) # <br> 
    main.h1.span.find('span', class_='lcDMY').append(soup2.new_tag('br')) # <br>
    for elem in main.find_all('h1'): # nechceme nadpisy, meníme na div
        elem.name = 'div'
    for elem in main.find_all('h2'): # nechceme nadpisy, meníme na div
        elem.name = 'div'

    # td1 bude obsahovať: datum (lcDMY) a deň v týždni (lcWD)
    td1 = soup1.new_tag('td', id=date_string) # id='2023-01-01' napríklad. využije sa pri naDnes navigácii
    td1.append( main.find('span', class_='lcDMY') ) # appending is also moving...   
    td1.append( main.find('span', class_='lcWD') ) # appending is also moving...
    new_table_row.append(td1) # <tr><td>    

    # td2 bude obsahovať: meniny (lcND) a zvyšok main soup.
    td2 = soup1.new_tag('td')
    td2.append( main.find('span', class_='lcND') ) # appending is also moving...
    for elem in main.find_all('div', class_='lcHEAD'):
        td2.append(elem)
        
    new_table_row.append(td2) # <tr><td>    

c:\Python\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [27]:
# final cleaning (size reducing)

# removing html comments <!-- -->
comments = soup1.find_all(string = lambda x: isinstance(x, Comment))
for comment in comments:
    comment.extract()

# removing tags with no content https://stackoverflow.com/questions/33500888/how-to-remove-tags-that-have-no-content
for x in soup1.find_all():
    if (len(x.get_text(strip=True)) == 0) and (x.name not in ['br', 'img', 'meta', 'link', 'script']):
        x.extract()

# soup1.smooth() # optionally

with open('onepage.html', mode='w', encoding='utf-8') as f_out:
    f_out.write(str(soup1))

In [ ]:
# done.